## Intro to Facial Recognition With OpenCV

### Prerequisites
* Python3 and OpenCV installed
  * [Install on Mac](https://www.learnopencv.com/install-opencv3-on-macos/)
  * [Install on Windows](https://www.learnopencv.com/install-opencv3-on-windows/)
* Numpy and jupyter installed
  * pip3 install numpy jupyter
* A working webcam

### Setup
* Create 2 folders: _dataset_ and _trainer_

### Imports

In [2]:
import cv2
import numpy as np
import os
import json
import random
from skimage import feature

### Constants and CV configs

In [3]:
# Camera - This will activate your camera
cam = cv2.VideoCapture(0)
cam.set(3, 640) # set video width
cam.set(4, 480) # set video height
font = cv2.FONT_HERSHEY_SIMPLEX # Font for text on image

# Face Classification Features
face_cascade = cv2.CascadeClassifier('./classifiers/haar_frontal_face.xml')

# Training Data Path
dataset_path = 'targets'

### Face Data Gathering

This will take 30 pictures of the face you'd like to recognize. They will be labeled with the name you give it in the prompt.

In [4]:
# Known Faces
known_faces = {} # Image Id => Person's name

face_name = input('\n Please enter your name and press <return> ==>  ')
face_id = random.randint(1, 1000)
known_faces[face_id] = face_name

sample_count = 0

# Take 30 snapshots of face for training
while(True):
    ret, img = cam.read() # capture an image
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # convert to gray so its easier to process
    
    # use Haar features to find faces in the image. 
    found_faces = face_cascade.detectMultiScale(gray_img, scaleFactor=1.2, minNeighbors=5, minSize=(20, 20))
    
    # Found faces is returned as the x, y coord of the upper left corner of the face and w, h of the face boundary
    for(x, y, w, h) in found_faces:
        cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2) # draw a blue rectangle on the color image
        sample_count += 1 # keep track of how many shots we took
        
        # Save the slice of the grayscale image that is the face in the dataset folder
        cv2.imwrite("./"+ dataset_path +"/" + face_name + "." + str(face_id) + "." + str(sample_count) + ".jpg", gray_img[y:y+h, x:x+w])
    
    # Show us the image so we can adjust position as necessary
    cv2.imshow('frame', img)
    
    k = cv2.waitKey(100) & 0xff # Press 'ESC' for exiting video
    if k == 27:
        break
    elif sample_count >= 30: # Take 30 face sample and stop video
         break
# Stop capturing the image
cam.release()
cv2.destroyAllWindows()

with open('known_faces.json', 'w') as outfile:
    json.dump(known_faces, outfile)


 Please enter your name and press <return> ==>  1


### Use Training Images to Create LBP Histogram

The LBP Histogram is essentially the numerical signature for your face. This is the baseline that the camera will compare your video feed to.

In [3]:
recognizer = cv2.face.LBPHFaceRecognizer_create()

known_faces = {}

def getImagesAndLabels(path):
    imagePaths = [os.path.join(path,f) for f in os.listdir(path)]     
    faceSamples=[]
    ids = []
    for imagePath in imagePaths:
        image = cv2.imread(imagePath)
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        img_numpy = np.array(gray_image,'uint8')
        face_name = (os.path.split(imagePath)[-1].split(".")[0])
        id = int((os.path.split(imagePath)[-1].split(".")[1]))
        known_faces[id] = face_name
        faces = face_cascade.detectMultiScale(img_numpy)
        for (x,y,w,h) in faces:
            faceSamples.append(img_numpy[y:y+h,x:x+w])
            ids.append(id)
    return faceSamples,ids

faces,ids = getImagesAndLabels('targets')
recognizer.train(faces, np.array(ids))
recognizer.write('trainer/trainer.yml')

# Print an example of what a face looks like in the context of Local Binary Pattern
# image = cv2.imread([os.path.join(dataset_path,f) for f in os.listdir(dataset_path)][2])
# gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
# features = feature.local_binary_pattern(gray_image, 10, 5, method="default")
# cv2.imwrite("./lbp_image.png", features.astype("uint8"))

# with open('known_faces.json', 'w') as outfile:
#     json.dump(known_faces, outfile)

## Detect and Identify Faces

You may need to restart the notebook to make this work. This opens a camera feed and detects _any face_ and turns that into a histogram and compares this histogram to all the histograms the recongizer knows about from _trainer.yml_. The closest result using the Euclidean difference (square root of sum of squares) within reason and assigns the label accordingly.

In [4]:
import cv2
import json

recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read('trainer/trainer.yml')

# Camera - This will activate your camera
cam = cv2.VideoCapture(0)
cam.set(3, 640) # set video width
cam.set(4, 480) # set video height
font = cv2.FONT_HERSHEY_SIMPLEX # Font for text on image

with open('known_faces.json') as infile:
    known_faces = json.load(infile)
    
# Face Classification Features
face_cascade = cv2.CascadeClassifier('./classifiers/haar_frontal_face.xml')

while(True):
    ret, img = cam.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=5, minSize=(20, 20))
    
    for(x, y, w, h) in faces:
        cv2.rectangle(img, (x, y), (x+w, y+h), (0, 225, 0), 2)
        id, confidence = recognizer.predict(gray[y:y+h, x:x+w])
        if(confidence < 100):
            id = known_faces[str(id)]
            confidence = " {0}%".format(round(100-confidence))
        else:
            id = "unk"
            confidence = " {0}%".format(round(100-confidence))
        
        cv2.putText(img, str(id), (x+5, y-5), font, 1, (255, 255, 255), 2)
        cv2.putText(img, str(confidence), (x+5, y+h-5), font, 1, (255, 255, 0), 1)

    cv2.imshow('frame', img)
        
    k = cv2.waitKey(100) & 0xff # Press 'ESC' for exiting video
    if k == 27:
        break

cam.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 